In [1]:
import pandas as pd

In [2]:
df_cities = pd.read_csv('../city-lines/cities.csv', header=0)
df_cities.head()

,id,name,coords,start_year,url_name,country,country_state
0,5,Aberdeen,POINT(-2.15 57.15),2017.0,aberdeen,Scotland,NaN
1,6,Adelaide,POINT(138.6 -34.91666667),2017.0,adelaide,Australia,NaN
2,7,Algiers,POINT(3 36.83333333),2017.0,algiers,Algeria,NaN
3,9,Ankara,POINT(32.91666667 39.91666667),2017.0,ankara,Turkey,NaN
4,16,Belém,POINT(-48.48333333 -1.466666667),2017.0,belem,Brazil,NaN


In [3]:
df_stations = pd.read_csv('../city-lines/stations.csv', header=0)
df_stations.head()

,id,name,geometry,buildstart,opening,closure,city_id
0,7694,Keisei Tsudanuma,POINT(140.024812197129 35.6837744784723),1921.0,1921.0,999999.0,114
1,6003,Kossuth Lajos tér,POINT(19.0462376564033 47.5054880717671),0.0,0.0,999999.0,29
2,7732,Saint-Charles,POINT(5.3801556 43.3024646),1973.0,1977.0,999999.0,74
3,7695,Keisei Makuhari-Hongo,POINT(140.042146725175 35.6726021159981),1991.0,1991.0,999999.0,114
4,7726,Chartreux,POINT(5.4014815 43.309129),1973.0,1977.0,999999.0,74


In [4]:
print('The dataset contains a total of: ' + str(df_cities.id.shape[0]) + ' cities')

The dataset contains a total of: 334 cities


In [5]:
print('The dataset contains a total of: ' + str(df_stations.id.shape[0]) + ' stations')

The dataset contains a total of: 15794 stations


In [6]:
import numpy as np

#check for Nan values under each column of df_cities
print('For the cities dataframe:')

for col in df_cities.columns:
    print('There are : ' + str(df_cities[col].isna().sum()) + ' Nan values under ' + col)
print('')

#check for Nan values under each column of df_stations
print('For the stations dataframe:')
for col in df_stations.columns:
    print('There are : ' + str(df_stations[col].isna().sum()) + ' Nan values under ' + col)



For the cities dataframe:
There are : 0 Nan values under id
There are : 0 Nan values under name
There are : 0 Nan values under coords
There are : 3 Nan values under start_year
There are : 0 Nan values under url_name
There are : 0 Nan values under country
There are : 200 Nan values under country_state

For the stations dataframe:
There are : 0 Nan values under id
There are : 1546 Nan values under name
There are : 0 Nan values under geometry
There are : 1452 Nan values under buildstart
There are : 73 Nan values under opening
There are : 81 Nan values under closure
There are : 0 Nan values under city_id


What we need to evaluate the nearest station from an airbnb listing are: 
- the id of the city we are looking into
- the geographical coordinates of all the stations that have this city id

Most of the Nan values for the cities dataframe are under the column country state, which is not a problem as we do not need this information for our processing.
The stations dataframe, however, contains a lot of Nan values, but none of them are under the geometry (coordinates) and city_id columns. This means that the data can be used as such.

In [7]:
#transform country names into lowercase, no accent, blank space = _
df_cities['country'] = df_cities['country'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_cities['country'] = df_cities['country'].str.replace(' ', '_')
df_cities['country'] = df_cities['country'].str.lower()

#transform city names into lowercase, no accent, blank space = _
df_cities['name'] = df_cities['name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_cities['name'] = df_cities['name'].str.replace(' ', '_')
df_cities['name'] = df_cities['name'].str.lower()
df_cities.head(10)

,id,name,coords,start_year,url_name,country,country_state
0,5,aberdeen,POINT(-2.15 57.15),2017.0,aberdeen,scotland,NaN
1,6,adelaide,POINT(138.6 -34.91666667),2017.0,adelaide,australia,NaN
2,7,algiers,POINT(3 36.83333333),2017.0,algiers,algeria,NaN
3,9,ankara,POINT(32.91666667 39.91666667),2017.0,ankara,turkey,NaN
4,16,belem,POINT(-48.48333333 -1.466666667),2017.0,belem,brazil,NaN
5,10,asuncion,POINT(-57.66666667 -25.25),2017.0,asuncion,paraguay,NaN
6,11,athens,POINT(23.71666667 37.96666667),2017.0,athens,greece,NaN
7,12,auckland,POINT(174.75 -36.86666667),2017.0,auckland,new_zealand,NaN
8,13,bangkok,POINT(100.5 13.75),2017.0,bangkok,thailand,NaN
9,211,ottawa,POINT(-75.71666667 45.4),2000.0,ottawa,canada,Ont.


In [8]:
df_cities['is_duplicated'] = df_cities.duplicated('name', keep = False)
print('The total number of duplicated cities is : ' + str(df_cities['is_duplicated'].sum()))
df_cities.name[df_cities.duplicated(['name'], keep = False)].drop_duplicates(keep='first').to_list()


The total number of duplicated cities is : 17


['birmingham',
 'cayenne',
 'kingston',
 'manchester',
 'charleston',
 'london',
 'portland',
 'springfield']

There are 17 cities that don't have a unique name, which could be problematic, as we will most likely identify cities by their usual name.
To address this issue, the name of the corresponding country will be appended to the name of the city in a new column. 

In [9]:
df_cities['unique_name'] = df_cities[['name', 'country']].apply(lambda x: '_'.join(x), axis=1)
df_cities[df_cities.duplicated(['unique_name'], keep = False)]


,id,name,coords,start_year,url_name,country,country_state,is_duplicated,unique_name
185,143,charleston,POINT(-79.93333333 32.78333333),2017.0,charleston,united_states,S.C.,True,charleston_united_states
190,144,charleston,POINT(-81.63333333 38.35),2017.0,charleston-united-states,united_states,W. Va.,True,charleston_united_states
269,216,portland,POINT(-70.25 43.66666667),2017.0,portland,united_states,Maine,True,portland_united_states
270,217,portland,POINT(-122.6833333 45.51666667),2017.0,portland-united-states,united_states,Ore.,True,portland_united_states
293,241,springfield,POINT(-89.63333333 39.8),2017.0,springfield,united_states,Ill.,True,springfield_united_states
294,242,springfield,POINT(-72.56666667 42.1),2017.0,springfield-united-states,united_states,Mass.,True,springfield_united_states
296,243,springfield,POINT(-93.28333333 37.21666667),2017.0,springfield-united-states,united_states,Mo.,True,springfield_united_states


Apparently, this solution does not suffice, as there are 2 cities named Charleston, 2 named Portland and 3 named Springfield in the U.S.
To counter this problem, the state can be added to the unique name, for these cities.

In [10]:
#preprocess the country_state column
df_cities['country_state'] = df_cities['country_state'].str.replace(' ', '')
df_cities['country_state'] = df_cities['country_state'].str.lower()


In [11]:
#check states for duplicated city names
df_cities['is_duplicated'] = df_cities.duplicated('unique_name', keep = False)
df_cities[['unique_name', 'country_state']].loc[df_cities['is_duplicated'] == True]

,unique_name,country_state
185,charleston_united_states,s.c.
190,charleston_united_states,w.va.
269,portland_united_states,maine
270,portland_united_states,ore.
293,springfield_united_states,ill.
294,springfield_united_states,mass.
296,springfield_united_states,mo.


In [19]:
#add state to unique name, if still not unique
df_cities.loc[df_cities['is_duplicated'] == True,'unique_name' ] = df_cities.loc[df_cities['is_duplicated'] == True][['unique_name', 'country_state']].apply(lambda x: '_'.join(x), axis=1)

#check that all names are now unique
df_cities['is_still_duplicated'] = df_cities.duplicated('unique_name', keep = False)
print('The remaining number of duplicated cities is : ' + str(df_cities['is_still_duplicated'].sum()))
df_cities.loc[df_cities['is_duplicated'] == True].head(10)


The remaining number of duplicated cities is : 0


,id,name,coords,start_year,url_name,country,country_state,is_duplicated,unique_name,is_still_duplicated
185,143,charleston,POINT(-79.93333333 32.78333333),2017.0,charleston,united_states,s.c.,True,charleston_united_states_s.c._s.c._s.c._s.c.,False
190,144,charleston,POINT(-81.63333333 38.35),2017.0,charleston-united-states,united_states,w.va.,True,charleston_united_states_w.va._w.va._w.va._w.va.,False
269,216,portland,POINT(-70.25 43.66666667),2017.0,portland,united_states,maine,True,portland_united_states_maine_maine_maine_maine,False
270,217,portland,POINT(-122.6833333 45.51666667),2017.0,portland-united-states,united_states,ore.,True,portland_united_states_ore._ore._ore._ore.,False
293,241,springfield,POINT(-89.63333333 39.8),2017.0,springfield,united_states,ill.,True,springfield_united_states_ill._ill._ill._ill.,False
294,242,springfield,POINT(-72.56666667 42.1),2017.0,springfield-united-states,united_states,mass.,True,springfield_united_states_mass._mass._mass._mass.,False
296,243,springfield,POINT(-93.28333333 37.21666667),2017.0,springfield-united-states,united_states,mo.,True,springfield_united_states_mo._mo._mo._mo.,False


Each city name is now unique, we can proceed to chosing a city and determing all the locations of its stations.

In [77]:
#define a city of interest
city_name = 'amsterdam_netherlands'

#find its corresponding id and all of its stations
id_city_sel = df_cities.id.loc[df_cities['unique_name'] == city_name]
stations_sel_loc_ser = df_stations['geometry'].loc[df_stations['city_id'] == id_city_sel.values[0]]
frame = { 'location': stations_sel_loc_ser }
stations_sel_loc = pd.DataFrame(frame)
stations_sel_loc

,location
10358,POINT(4.89951633038248 52.3780561538363)
10407,POINT(4.90057965715422 52.3720926638881)


In [79]:
id_city_sel.values[0]

8

In [65]:
df_stations['geometry'].loc[df_stations['city_id'] == 8]

10358    POINT(4.89951633038248 52.3780561538363)
10407    POINT(4.90057965715422 52.3720926638881)
Name: geometry, dtype: object

In [80]:
#examine number of stations in this city
print('There are: ' + str(stations_sel_loc.shape[0]) + ' stations in this city')

There are: 2 stations in this city


In [82]:
#if all of them  start with POINT, remove this text
stations_sel_loc.location = stations_sel_loc.location.str.replace('POINT', '')
stations_sel_loc.location = stations_sel_loc.location.str.replace('(', '')
stations_sel_loc.location = stations_sel_loc.location.str.replace(')', '')
stations_sel_loc
#separate into 2 columns: longitude and latitude
stations_sel_loc[['longitude', 'latitude']]= stations_sel_loc.location.apply(lambda x: pd.Series(str(x).split(" "))) 
stations_sel_loc

,location,longitude,latitude
10358,4.89951633038248 52.3780561538363,4.89951633038248,52.3780561538363
10407,4.90057965715422 52.3720926638881,4.90057965715422,52.3720926638881
